In [1]:
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()
df = spark.read.option("header",True).csv("data/Programme_Wise_Graduated_Student_Category_Stats-17-18.csv")
df.printSchema()

root
 |-- Programme: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Others: string (nullable = true)
 |-- Total: string (nullable = true)



In [2]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
df2 = df.withColumn("total_new",col("Total").cast(IntegerType()))

In [3]:
df2.groupBy("programme").sum("total_new").show(truncate=False)

+---------------------------------------------------------+--------------+
|programme                                                |sum(total_new)|
+---------------------------------------------------------+--------------+
|Total                                                    |39            |
|MPhil in Sports Science                                  |2             |
|null                                                     |15            |
|PhD in Sports Science                                    |1             |
|5 yr Intd. MSc in Agriculture, Rural & Tribal Development|9             |
|Diploma in Agriculture & Rural Development               |9             |
|PhD in Agriculture, Rural & Tribal Development           |2             |
|Intd. MPhil-PhD in Special Education                     |1             |
+---------------------------------------------------------+--------------+



In [4]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
acc = spark.sparkContext.accumulator(0)
#spark.udf.register("recreate_table", repeatName, StringType())

@udf(returnType=StringType()) 
def repeatName(str):
    global acc
    #return str
    print (str)
    if (not str):
        return acc
    else:
        acc = str
        return acc

df3 = df2.withColumn("Name", repeatName(col("Programme")))
df3.show(truncate=False)

+---------------------------------------------------------+--------+----+------+------+-----+---------+---------------------------------------------------------+
|Programme                                                |Category|Male|Female|Others|Total|total_new|Name                                                     |
+---------------------------------------------------------+--------+----+------+------+-----+---------+---------------------------------------------------------+
|MPhil in Sports Science                                  |GENERAL |1   |1     |0     |2    |2        |MPhil in Sports Science                                  |
|null                                                     |OBC     |0   |1     |0     |1    |1        |MPhil in Sports Science                                  |
|null                                                     |SC      |3   |0     |0     |3    |3        |MPhil in Sports Science                                  |
|PhD in Sports Science      

In [5]:
df4 = df3.select("Name", "Category", "Total")
df4.show(truncate=False)

+---------------------------------------------------------+--------+-----+
|Name                                                     |Category|Total|
+---------------------------------------------------------+--------+-----+
|MPhil in Sports Science                                  |GENERAL |2    |
|MPhil in Sports Science                                  |OBC     |1    |
|MPhil in Sports Science                                  |SC      |3    |
|PhD in Sports Science                                    |OBC     |1    |
|Diploma in Agriculture & Rural Development               |OBC     |9    |
|Diploma in Agriculture & Rural Development               |ST      |8    |
|5 yr Intd. MSc in Agriculture, Rural & Tribal Development|GENERAL |9    |
|5 yr Intd. MSc in Agriculture, Rural & Tribal Development|OBC     |1    |
|5 yr Intd. MSc in Agriculture, Rural & Tribal Development|SC      |1    |
|PhD in Agriculture, Rural & Tribal Development           |GENERAL |2    |
|Intd. MPhil-PhD in Speci

In [6]:
df_dept = spark.read.option("header",True).csv("data/Programme_List.csv")
df_dept.show()

+--------------------+-------------+-------------+-------------+------------+----------+--------+--------------+--------------+
|           Programme|         Code|Duration (Yr)|Maximum Marks|      Campus|Department|  Degree|Faculty Center|No of Students|
+--------------------+-------------+-------------+-------------+------------+----------+--------+--------------+--------------+
|Integrated 5 Year...|      SAN-MA5|        5 - 6|          100|Belur Campus|      SANS|5YrIntMA|          null|            73|
|      MSc in Physics|         PHY1|        2 - 2|          100|Belur Campus|      PHYS|     MSC|          null|            63|
|MSc in Big Data A...|          BDA|        2 - 3|          100|Belur Campus|    COMPSC|     MSC|          null|            37|
|MSc in Computer S...|          CS1|        2 - 3|          100|Belur Campus|    COMPSC|     MSC|          null|            24|
|Integrated MPhil-...|SANS-Intd.PHD|       5 - 10|          100|Belur Campus|      SANS|     PhD|       

In [7]:
df_dept_stat = df4.join(df_dept,df4.Name ==  df_dept.Programme,"inner")

In [8]:
df_data = df_dept_stat.select("Department", "Programme", "Category","Total")
df_data.show()

+----------+--------------------+--------+-----+
|Department|           Programme|Category|Total|
+----------+--------------------+--------+-----+
|    SSYOGA|PhD in Sports Sci...|     OBC|    1|
|    SSYOGA|MPhil in Sports S...|      SC|    3|
|    SSYOGA|MPhil in Sports S...|     OBC|    1|
|    SSYOGA|MPhil in Sports S...| GENERAL|    2|
+----------+--------------------+--------+-----+



In [9]:
df_data2 = df_data.withColumn("total_new",col("Total").cast(IntegerType()))
df_data2.coalesce(1).write.option("header",True).csv('out/17-18/tmp')
df_data3 = spark.read.option("header",True).csv("out/17-18/tmp/*.csv")

#spark.udf.register("recreate_table", repeatName, StringType())
df_trimmed = df_data3.withColumn("total",col("total_new").cast(IntegerType()))
df_trimmed.show()

+----------+--------------------+--------+-----+---------+
|Department|           Programme|Category|total|total_new|
+----------+--------------------+--------+-----+---------+
|    SSYOGA|PhD in Sports Sci...|     OBC|    1|        1|
|    SSYOGA|MPhil in Sports S...|      SC|    3|        3|
|    SSYOGA|MPhil in Sports S...|     OBC|    1|        1|
|    SSYOGA|MPhil in Sports S...| GENERAL|    2|        2|
+----------+--------------------+--------+-----+---------+



In [10]:
df_Gen_data = df_trimmed.filter(df_trimmed.Category == "GENERAL")
df_Gen_Ews_data = df_trimmed.filter(df_trimmed.Category == "GENERAL-EWS")
df_Sc_data = df_trimmed.filter(df_trimmed.Category == "SC")
df_St_data = df_trimmed.filter(df_trimmed.Category == "ST")
df_Obc_data = df_trimmed.filter(df_trimmed.Category == "OBC")

#df_Gen_data.show(truncate=False)
#df_Gen_Ews_data.show()
#df_Sc_data.show()
#df_St_data.show()
#df_Obc_data.show()

In [11]:
df_Gen = df_Gen_data.groupBy("Department").sum("total")
df_Gen_Ews = df_Gen_Ews_data.groupBy("Department").sum("total")
df_Sc = df_Sc_data.groupBy("Department").sum("total")
df_St = df_St_data.groupBy("Department").sum("total")
df_Obc = df_Obc_data.groupBy("Department").sum("total")
df_St.show(truncate=False)

+----------+----------+
|Department|sum(total)|
+----------+----------+
+----------+----------+



In [12]:
df_Gen.coalesce(1).write.csv('out/17-18/Gen.csv')

In [13]:
df_Gen_Ews.coalesce(1).write.csv('out/17-18/Gen_Ews.csv')
df_Sc.coalesce(1).write.csv('out/17-18/Sc.csv')
df_St.coalesce(1).write.csv('out/17-18/St.csv')
df_Obc.coalesce(1).write.csv('out/17-18/Obc.csv')